In [1]:
import torch
import config
from cnn_utils import (
    get_model_instance_segmentation,
    collate_fn,
    get_transform,
    myOwnDataset,
)

In [2]:
print("Torch version:", torch.__version__)

# create own Dataset
my_dataset = myOwnDataset(
    root=config.train_data_dir, annotation=config.train_coco, transforms=get_transform()
)

test = myOwnDataset(
    root=config.test_data_dir, annotation=config.test_coco, transforms=get_transform()
)

# own DataLoader
data_loader = torch.utils.data.DataLoader(
    my_dataset,
    batch_size=config.train_batch_size,
    shuffle=config.train_shuffle_dl,
    num_workers=config.num_workers_dl,
    collate_fn=collate_fn,
)

#test Dataloader
test_loader = torch.utils.data.DataLoader(
test, batch_size=1, shuffle=False, num_workers=0,
collate_fn=collate_fn)


# select device (whether GPU or CPU)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print('Running on ', device)

# DataLoader is iterable over Dataset
for imgs, annotations in data_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)


model = get_model_instance_segmentation(config.num_classes)

# move model to the right device
model.to(device)

# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params, lr=config.lr, momentum=config.momentum, weight_decay=config.weight_decay
)

len_dataloader = len(data_loader)

# Training
for epoch in range(config.num_epochs):
    print(f"Epoch: {epoch}/{config.num_epochs}")
    model.train()
    i = 0
    for imgs, annotations in data_loader:
        i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        print(f"Iteration: {i}/{len_dataloader}, Loss: {losses}")

Torch version: 1.10.0
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Running on  cuda
[{'boxes': tensor([[ 56.,  97.,  99., 121.],
        [ 76., 123., 119., 142.],
        [ 52., 148.,  97., 168.]], device='cuda:0'), 'labels': tensor([1, 1, 3], device='cuda:0'), 'image_id': tensor([154], device='cuda:0'), 'area': tensor([1032.,  817.,  900.], device='cuda:0'), 'iscrowd': tensor([0, 0, 0], device='cuda:0')}]
[{'boxes': tensor([[ 64.,  72., 136., 147.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([256], device='cuda:0'), 'area': tensor([5400.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[ 60.,   4., 162., 221.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([96], device='cuda:0'), 'area': tensor([22134.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'bo

[{'boxes': tensor([[ 53.,  47., 101., 107.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0'), 'image_id': tensor([190], device='cuda:0'), 'area': tensor([2880.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[139.,  92., 180., 143.],
        [ 25., 104.,  66., 152.],
        [ 12., 110.,  26., 142.]], device='cuda:0'), 'labels': tensor([2, 2, 2], device='cuda:0'), 'image_id': tensor([160], device='cuda:0'), 'area': tensor([2091., 1968.,  448.], device='cuda:0'), 'iscrowd': tensor([0, 0, 0], device='cuda:0')}]
[{'boxes': tensor([[ 87., 105., 175., 143.]], device='cuda:0'), 'labels': tensor([0], device='cuda:0'), 'image_id': tensor([280], device='cuda:0'), 'area': tensor([3344.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[ 74., 137.,  92., 172.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0'), 'image_id': tensor([240], device='cuda:0'), 'area': tensor([630.], device='cuda:0'), 'iscrowd': tensor([0]

[{'boxes': tensor([[115., 171., 130., 196.],
        [ 64., 137.,  77., 156.]], device='cuda:0'), 'labels': tensor([3, 3], device='cuda:0'), 'image_id': tensor([289], device='cuda:0'), 'area': tensor([375., 247.], device='cuda:0'), 'iscrowd': tensor([0, 0], device='cuda:0')}]
[{'boxes': tensor([[ 71.,  88., 127., 122.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([74], device='cuda:0'), 'area': tensor([1904.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[115.,  15., 171.,  73.]], device='cuda:0'), 'labels': tensor([3], device='cuda:0'), 'image_id': tensor([1], device='cuda:0'), 'area': tensor([3248.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[ 52.,  52., 117., 112.]], device='cuda:0'), 'labels': tensor([3], device='cuda:0'), 'image_id': tensor([282], device='cuda:0'), 'area': tensor([3900.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[42., 56., 

[{'boxes': tensor([[ 31.,   0., 215., 120.]], device='cuda:0'), 'labels': tensor([0], device='cuda:0'), 'image_id': tensor([83], device='cuda:0'), 'area': tensor([22080.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[  0.,  31.,  38., 123.],
        [ 48.,  10.,  89.,  89.],
        [ 87.,   7., 119.,  76.],
        [167., 102., 207., 160.],
        [153.,  58., 183., 129.],
        [122.,  38., 148.,  71.]], device='cuda:0'), 'labels': tensor([2, 2, 2, 3, 2, 3], device='cuda:0'), 'image_id': tensor([310], device='cuda:0'), 'area': tensor([3496., 3239., 2208., 2320., 2130.,  858.], device='cuda:0'), 'iscrowd': tensor([0, 0, 0, 0, 0, 0], device='cuda:0')}]
[{'boxes': tensor([[ 60., 104., 133., 128.]], device='cuda:0'), 'labels': tensor([3], device='cuda:0'), 'image_id': tensor([210], device='cuda:0'), 'area': tensor([1752.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[ 70.,  22., 177.,  73.]], device='cuda:0'), 'labe

[{'boxes': tensor([[ 41.,  89.,  69., 112.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0'), 'image_id': tensor([305], device='cuda:0'), 'area': tensor([644.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[ 49.,  62., 121., 162.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([34], device='cuda:0'), 'area': tensor([7200.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[ 70.,  78., 120., 162.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([316], device='cuda:0'), 'area': tensor([4200.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[102.,  21., 168., 145.],
        [163., 153., 215., 180.]], device='cuda:0'), 'labels': tensor([0, 0], device='cuda:0'), 'image_id': tensor([47], device='cuda:0'), 'area': tensor([8184., 1404.], device='cuda:0'), 'iscrowd': tensor([0, 0], device='cuda:0')}]
[{'boxes': tensor([[ 14.,  6

...

Iteration: 44/306, Loss: 0.44384562969207764
Iteration: 45/306, Loss: 0.20793378353118896
Iteration: 46/306, Loss: 0.1405142843723297
Iteration: 47/306, Loss: 0.24411319196224213
Iteration: 48/306, Loss: 0.3842361271381378
Iteration: 49/306, Loss: 0.07079295814037323
Iteration: 50/306, Loss: 0.07486997544765472
Iteration: 51/306, Loss: 0.47614067792892456
Iteration: 52/306, Loss: 0.3623424172401428
Iteration: 53/306, Loss: 0.9985923171043396
Iteration: 54/306, Loss: 0.052922360599040985
Iteration: 55/306, Loss: 0.08527572453022003
Iteration: 56/306, Loss: 0.15755164623260498
Iteration: 57/306, Loss: 0.12749643623828888
Iteration: 58/306, Loss: 0.0950857549905777
Iteration: 59/306, Loss: 0.4631500244140625
Iteration: 60/306, Loss: 0.3670231103897095
Iteration: 61/306, Loss: 0.3475513458251953
Iteration: 62/306, Loss: 0.04874785989522934
Iteration: 63/306, Loss: 0.07519909739494324
Iteration: 64/306, Loss: 0.24172689020633698
Iteration: 65/306, Loss: 0.11947055160999298
Iteration: 66/306

Iteration: 226/306, Loss: 0.6303039789199829
Iteration: 227/306, Loss: 0.266403466463089
Iteration: 228/306, Loss: 0.8777534365653992
Iteration: 229/306, Loss: 0.2829878628253937
Iteration: 230/306, Loss: 0.3968963027000427
Iteration: 231/306, Loss: 0.5838194489479065
Iteration: 232/306, Loss: 0.28906527161598206
Iteration: 233/306, Loss: 0.22038839757442474
Iteration: 234/306, Loss: 0.19461725652217865
Iteration: 235/306, Loss: 0.30312979221343994
Iteration: 236/306, Loss: 1.0023245811462402
Iteration: 237/306, Loss: 0.5923013687133789
Iteration: 238/306, Loss: 0.18481867015361786
Iteration: 239/306, Loss: 0.17394402623176575
Iteration: 240/306, Loss: 0.16355064511299133
Iteration: 241/306, Loss: 0.19788791239261627
Iteration: 242/306, Loss: 0.341844767332077
Iteration: 243/306, Loss: 0.23039934039115906
Iteration: 244/306, Loss: 0.2798570394515991
Iteration: 245/306, Loss: 0.1343916803598404
Iteration: 246/306, Loss: 0.012730647809803486
Iteration: 247/306, Loss: 0.0564860962331295
I

Iteration: 103/306, Loss: 0.5699602961540222
Iteration: 104/306, Loss: 0.2122509777545929
Iteration: 105/306, Loss: 0.15563459694385529
Iteration: 106/306, Loss: 0.10743927955627441
Iteration: 107/306, Loss: 0.9053819179534912
Iteration: 108/306, Loss: 0.14967679977416992
Iteration: 109/306, Loss: 0.12277568131685257
Iteration: 110/306, Loss: 0.4598568379878998
Iteration: 111/306, Loss: 0.19253472983837128
Iteration: 112/306, Loss: 0.16669109463691711
Iteration: 113/306, Loss: 0.09232039004564285
Iteration: 114/306, Loss: 0.7769006490707397
Iteration: 115/306, Loss: 0.20090289413928986
Iteration: 116/306, Loss: 0.3361425995826721
Iteration: 117/306, Loss: 0.36127769947052
Iteration: 118/306, Loss: 0.36424174904823303
Iteration: 119/306, Loss: 0.5355062484741211
Iteration: 120/306, Loss: 0.211943119764328
Iteration: 121/306, Loss: 0.1618557572364807
Iteration: 122/306, Loss: 0.10393545776605606
Iteration: 123/306, Loss: 0.15912124514579773
Iteration: 124/306, Loss: 0.04033622890710831
I

Iteration: 284/306, Loss: 0.14694249629974365
Iteration: 285/306, Loss: 0.7846890091896057
Iteration: 286/306, Loss: 0.40820249915122986
Iteration: 287/306, Loss: 0.10610362887382507
Iteration: 288/306, Loss: 0.3716142475605011
Iteration: 289/306, Loss: 0.12633134424686432
Iteration: 290/306, Loss: 0.16675931215286255
Iteration: 291/306, Loss: 0.05529427528381348
Iteration: 292/306, Loss: 0.1642695963382721
Iteration: 293/306, Loss: 0.5056350231170654
Iteration: 294/306, Loss: 0.7660959959030151
Iteration: 295/306, Loss: 0.26870161294937134
Iteration: 296/306, Loss: 0.466653436422348
Iteration: 297/306, Loss: 1.0207377672195435
Iteration: 298/306, Loss: 0.26435261964797974
Iteration: 299/306, Loss: 0.3078854978084564
Iteration: 300/306, Loss: 0.18894393742084503
Iteration: 301/306, Loss: 0.2948247492313385
Iteration: 302/306, Loss: 0.6463038921356201
Iteration: 303/306, Loss: 0.28473764657974243
Iteration: 304/306, Loss: 1.1179488897323608
Iteration: 305/306, Loss: 0.3789626657962799
I

Iteration: 161/306, Loss: 0.1602451652288437
Iteration: 162/306, Loss: 0.2548167109489441
Iteration: 163/306, Loss: 0.21647630631923676
Iteration: 164/306, Loss: 0.3430529832839966
Iteration: 165/306, Loss: 0.06602606922388077
Iteration: 166/306, Loss: 0.15227000415325165
Iteration: 167/306, Loss: 0.2299133688211441
Iteration: 168/306, Loss: 0.47380173206329346
Iteration: 169/306, Loss: 0.9097393751144409
Iteration: 170/306, Loss: 0.22431698441505432
Iteration: 171/306, Loss: 0.1229451447725296
Iteration: 172/306, Loss: 0.06952362507581711
Iteration: 173/306, Loss: 0.07320340722799301
Iteration: 174/306, Loss: 0.24880404770374298
Iteration: 175/306, Loss: 0.30147671699523926
Iteration: 176/306, Loss: 0.2933177053928375
Iteration: 177/306, Loss: 0.29421988129615784
Iteration: 178/306, Loss: 0.6127868294715881
Iteration: 179/306, Loss: 0.5566725730895996
Iteration: 180/306, Loss: 0.14877158403396606
Iteration: 181/306, Loss: 0.16869403421878815
Iteration: 182/306, Loss: 0.324656963348388

In [3]:
PATH = './frcnn_net_evaluate.pth'
torch.save(model.state_dict(), PATH)